In [1]:
# Asteroid is based on PyTorch and PyTorch-Lightning.
from torch import optim
from pytorch_lightning import Trainer

# We train the same model architecture that we used for inference above.
from asteroid.models import ConvTasNet

# In this example we use Permutation Invariant Training (PIT) and the SI-SDR loss.
from asteroid.losses import pairwise_neg_sisdr, PITLossWrapper

# MiniLibriMix is a tiny version of LibriMix (https://github.com/JorisCos/LibriMix),
# which is a free speech separation dataset.
from asteroid.data import LibriMix

# Asteroid's System is a convenience wrapper for PyTorch-Lightning.
from asteroid.engine import System

from custom_dataset_sandbox import CustomMixture



In [2]:
# This will automatically download MiniLibriMix from Zenodo on the first run.
# train_loader, val_loader = LibriMix.loaders_from_mini(task="sep_clean", batch_size=16)
# train_loader, val_loader = LibriMix.mini_from_download(task="sep_clean", {"num_workers": 8})
csv_path = "C:/Users/ZaknafeinII/Desktop/University_of_Iowa/4_Senior/Spring/ECE4890_SD/voiceseperation/ML_data_preparation/MiniLibriMix/metadata"
train_loader = CustomMixture(csv_path, "example_train")
val_loader = CustomMixture(csv_path, "example_train")

# train_loader = LibriMix("C:/Users/ZaknafeinII/Desktop/University_of_Iowa/4_Senior/Spring/ECE4890_SD/voiceseperation/ML_data_preparation/MiniLibriMix/metadata/train")
# val_loader = LibriMix("C:/Users/ZaknafeinII/Desktop/University_of_Iowa/4_Senior/Spring/ECE4890_SD/voiceseperation/ML_data_preparation/MiniLibriMix/metadata/val")
# Tell DPRNN that we want to separate to 2 sources.
model = ConvTasNet(n_src=2)

# PITLossWrapper works with any loss function.
loss = PITLossWrapper(pairwise_neg_sisdr, pit_from="pw_mtx")

optimizer = optim.Adam(model.parameters(), lr=1e-3)

system = System(model, optimizer, loss, train_loader, val_loader)

In [3]:
# from torch import cuda
# import torch
# print(cuda.device_count())
# print(torch.version.cuda)
# print(torch.cuda.is_available())

# # print(type(model))

In [5]:
# Train for 1 epoch using a single GPU. If you're running this on Google Colab,
# be sure to select a GPU runtime (Runtime → Change runtime type → Hardware accelarator).
# trainer = Trainer(max_epochs=1, gpus=0, auto_select_gpus=False)
trainer = Trainer()
trainer.fit(system)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name      | Type           | Params
---------------------------------------------
0 | model     | ConvTasNet     | 5.1 M 
1 | loss_func | PITLossWrapper | 0     
---------------------------------------------
5.1 M     Trainable params
0         Non-trainable params
5.1 M     Total params
20.202    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

sources.shape: torch.Size([2, 121920])


TypeError: Expected data to be int, Sequence or Mapping, but got CustomMixture